# Foundation Model Prompt Engineering: Text2Text Generation


In [2]:
import logging
import json
import time
import boto3

import matplotlib.pyplot as plt
import numpy as np
import requests
from tqdm.contrib.concurrent import thread_map

region = boto3.Session().region_name

logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())
url = "https://yyosweaa3e.execute-api.us-east-1.amazonaws.com/LATEST/HF"

In [3]:
def query_endpoint_with_json_payload(url, payload):
    response = requests.post(
        url,
        json=payload,
    )
    return response

def parse_response_multiple_texts(response):
    model_predictions = json.loads(json.dumps(response.json()))
    generated_text = model_predictions['generated_texts']
    return generated_text

### Advanced features

***
This model also supports many advanced parameters while performing inference. They include:

* **max_length:** Model generates text until the output length (which includes the input context length) reaches `max_length`. If specified, it must be a positive integer.
* **num_return_sequences:** Number of output sequences returned. If specified, it must be a positive integer.
* **num_beams:** Number of beams used in the greedy search. If specified, it must be integer greater than or equal to `num_return_sequences`.
* **no_repeat_ngram_size:** Model ensures that a sequence of words of `no_repeat_ngram_size` is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.
* **temperature:** Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If `temperature` -> 0, it results in greedy decoding. If specified, it must be a positive float.
* **early_stopping:** If True, text generation is finished when all beam hypotheses reach the end of sentence token. If specified, it must be boolean.
* **do_sample:** If True, sample the next word as per the likelihood. If specified, it must be boolean.
* **top_k:** In each step of text generation, sample from only the `top_k` most likely words. If specified, it must be a positive integer.
* **top_p:** In each step of text generation, sample from the smallest possible set of words with cumulative probability `top_p`. If specified, it must be a float between 0 and 1.
* **seed:** Fix the randomized state for reproducibility. If specified, it must be an integer.

We may specify any subset of the parameters mentioned above while invoking an endpoint. Next, we show an example of how to invoke endpoint with these arguments

***

## N-shot Learning via In-context Learning

Let's put in some example input text. You can put any text containing the task, the model returns the output of the accomplished task.

Payload must be a json

### A. Zero-shot Learning

#### Extract Q&A

In [7]:
context = """Once, a cunning fox saw a crow with a piece of cheese in its beak sitting on a branch. \
The fox devised a plan and flattered the crow, causing the crow to caw with delight, dropping the \
cheese which the fox quickly snatched up and ran away. The crow learned a valuable lesson and never trusted the fox again."""

question = 'who got cheated?'
# question = "what lesson did the crow learn?"
answer = 'crow'

prompt = f'Context: {context}<br>Question: {question}<br>Answer:'
logger.info(f'Prompt: {prompt}')

Prompt: Context: Once, a cunning fox saw a crow with a piece of cheese in its beak sitting on a branch. The fox devised a plan and flattered the crow, causing the crow to caw with delight, dropping the cheese which the fox quickly snatched up and ran away. The crow learned a valuable lesson and never trusted the fox again.<br>Question: who got cheated?<br>Answer:


In [8]:
payload = {"text_inputs":prompt, "max_length":50, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: crow


#### Sentiment Analysis/Classification

In [9]:
review = 'I hated the movie. Thoroughly disappointing for a sequel.'
# review = 'this product is fine so far'
sentiment = 'Sentiment (Good, Bad)'

prompt = f'Review: {review}\n{sentiment}:'
logger.info(f'Prompt: {prompt}')

Prompt: Review: I hated the movie. Thoroughly disappointing for a sequel.
Sentiment (Good, Bad):


In [10]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": 0.0, "seed": 321}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: Bad


#### Natural Language Inference (NLI)

In [11]:
location = "United States"
# location = "United Kingdom"
prompt = f"""The world cup has kicked off in Los Angeles, United States.
Based on the paragraph above can we conclude that: "The world cup takes place in {location}"?
"""

logger.info(f"Prompt: {prompt}")

Prompt: The world cup has kicked off in Los Angeles, United States.
Based on the paragraph above can we conclude that: "The world cup takes place in United States"?



In [12]:
payload = {"text_inputs":prompt, "max_length":50}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: yes


#### Text Generation

In [19]:
prompt = """
Title: "University has new facility coming up"
Given the above title of an imaginary article, imagine the article.
"""

logger.info(f"Prompt: {prompt}")

Prompt: 
Title: "University has new facility coming up"
Given the above title of an imaginary article, imagine the article.



In [20]:
payload = {"text_inputs":prompt, "max_length": 200}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: The University of the West Indies (UWI) has announced that it will be building a new sports facility on its St Augustine's campus. The new facility will be located on the St Augustine's campus and will be a multi-purpose sports complex. The new sports complex will be built on the St Augustine's campus and will be a multi-purpose sports complex. The new sports complex will be built on the St Augustine's campus and will be a multi-purpose sports complex. The new sports complex will be built on the St Augustine's campus and will be a multi-purpose sports complex. The new sports complex will be built on the St Augustine's campus and will be a multi-purpose sports complex. The new sports complex will be built on the St Augustine's campus and will be a multi-purpose sports complex. The new sports complex will be built on the St Augustine


#### Translation

In [23]:
prompt = "Translate to German:  My name is Arthur"

logger.info(f"Prompt: {prompt}")

Prompt: Translate to German:  My name is Arthur


In [24]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 0.2, "do_sample": False, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: Ich bin Arthur.


### Q&A

In [25]:
context = """
Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try restarting your iPhone. Press and hold the power button until you see the "slide to power off" option. Slide to power off, wait a few seconds, and then turn your iPhone back on.
Customer: Alright, I restarted it, but it's still not charging properly.
Agent: I see. It looks like we need to run a diagnostic test on your iPhone. Please visit the nearest Apple Store or authorized service provider to get your iPhone checked out.
Customer: Do I need to make an appointment?
Agent: Yes, it's always best to make an appointment beforehand so you don't have to wait in line. You can make an appointment online or by calling the Apple Store or authorized service provider.
Customer: Okay, will I have to pay for the repairs?
Agent: That depends on whether your iPhone is covered under warranty or not. If it is, you won't have to pay anything. However, if it's not covered under warranty, you will have to pay for the repairs.
Customer: How long will it take to get my iPhone back?
Agent: It depends on the severity of the issue, but it usually takes 1-2 business days.
Customer: Can I track the repair status online?
Agent: Yes, you can track the repair status online or by calling the Apple Store or authorized service provider.
Customer: Alright, thanks for your help.
Agent: No problem, happy to help. Is there anything else I can assist you with?
Customer: No, that's all for now.
Agent: Alright, have a great day and good luck with your iPhone!
"""

query = (
    "What are the customer and agent talking about?"
)

# "What troubleshooting steps were suggested to the customer to fix their iPhone charging issue?"
# "Was resetting the iPhone to its default settings able to solve the charging issue and battery draining?"
# "What steps can the customer take to make an appointment at the nearest Apple Store or authorized service provider?"
# "What is the overall sentiment and sentiment score of the conversation between the customer and the agent?"
# "identify any specific words, phrases, or context that influenced the {sentiment} sentiment."

prompt = f"""
Context: 
{context}

{query}
"""

logger.info(f'Prompt: {prompt}')

Prompt: 
Context: 

Customer: Hi there, I'm having a problem with my iPhone.
Agent: Hi! I'm sorry to hear that. What's happening?
Customer: The phone is not charging properly, and the battery seems to be draining very quickly. I've tried different charging cables and power adapters, but the issue persists.
Agent: Hmm, that's not good. Let's try some troubleshooting steps. Can you go to Settings, then Battery, and see if there are any apps that are using up a lot of battery life?
Customer: Yes, there are some apps that are using up a lot of battery.
Agent: Okay, try force quitting those apps by swiping up from the bottom of the screen and then swiping up on the app to close it.
Customer: I did that, but the issue is still there.
Agent: Alright, let's try resetting your iPhone's settings to their default values. This won't delete any of your data. Go to Settings, then General, then Reset, and then choose Reset All Settings.
Customer: Okay, I did that. What's next?
Agent: Now, let's try r

In [ ]:
payload = {"text_inputs":prompt, "max_length": 150, "temperature": 0.5, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

#### In Context Q&A

In [16]:
test_context = """The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) \
were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. \
They were descended from Norse (\"Norman\" comes from \"Norseman\") raiders and pirates from Denmark, \
Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. \
Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, \
their descendants would gradually merge with the Carolingian-based cultures of West Francia. \
The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, \
and it continued to evolve over the succeeding centuries."""
test_question = "In what country is Normandy located?"

prompt = f"""
Context: 
{test_context}

{test_question}
"""

logger.info(f'Prompt: {prompt}')

Prompt: 
Context: 
The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.

In what country is Normandy located?



In [17]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 1.2, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: France


### B. One-shot Learning

#### Text Summarisation

In [18]:
train_article = 'I love apples especially the large juicy ones. Apples are a great source of vitamins and fiber. An apple a day keeps the doctor away!'
train_summary = 'I love apples.'

test_article = 'I hate oranges especially the bitter ones. They are high in citric acid and they give me heart burns. Doctor suggests me to avoid them!'
test_summary = 'I hate oranges.'

In [19]:
prompt = f"""
article: {train_article}
summary: {train_summary}
--
article: {test_article}
summary:
"""

logger.info(f'Prompt: {prompt}')

Prompt: 
article: I love apples especially the large juicy ones. Apples are a great source of vitamins and fiber. An apple a day keeps the doctor away!
summary: I love apples.
--
article: I hate oranges especially the bitter ones. They are high in citric acid and they give me heart burns. Doctor suggests me to avoid them!
summary:



In [20]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": 0.5, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: I hate oranges. --


#### Natural Language Generation (NLG)

In [21]:
train_inp = 'name[The Punter], eat_type[Indian], price_range[cheap]'
train_out = 'The Punter provides Indian food in the cheap price range.'

test_inp = 'name[Blue Spice], eatType[coffee shop], price_range[expensive]'
test_out = 'Blue Spice is a coffee shop that is a bit expensive.'

prompt = (
    f"{train_inp} ==> {train_out}\n"
    f"--\n"
    f"{test_inp} ==>"
)

logger.info(f'Prompt: {prompt}')

Prompt: name[The Punter], eat_type[Indian], price_range[cheap] ==> The Punter provides Indian food in the cheap price range.
--
name[Blue Spice], eatType[coffee shop], price_range[expensive] ==>


In [22]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 0.5, "seed": 123, "num_beams": 3}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: Blue Spice is an expensive coffee shop.


#### Flip (Entity Extraction)

In [23]:
train_inp = 'The Punter provides Indian food in the cheap price range.'
train_out = 'name[The Punter], eat_type[Indian], price_range[cheap]'

test_inp = 'Blue Spice is a coffee shop that is a bit pricy.'
test_out = 'name[Blue Spice], eat_type[coffee shop], price_range[pricy]'

prompt = (
    f"""{train_inp} ==> {train_out}\n"""
    f"--\n"
    f"""{test_inp} ==>"""
)

logger.info(f'Prompt: {prompt}')

Prompt: The Punter provides Indian food in the cheap price range. ==> name[The Punter], eat_type[Indian], price_range[cheap]
--
Blue Spice is a coffee shop that is a bit pricy. ==>


In [24]:
payload = {"text_inputs":prompt, "max_length": 100, "temperature": 0.1, "do_sample": False, "seed": 123, "num_beams": 3}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: name[Blue Spice], eat_type[coffee shop], price[high] --


### C. Few-shot Learning

#### Entity Extraction

In [25]:
prompt = f"""
Extract main person:

s: John is playing basketball
p: John
##
s: Jeff and Phil are chatting about GAI. Phil has to run. He is in a rush
p: Phil
##
s: Max is older than Emma
p: Max
##
s: Susan misses the bus this morning but still get in time for her meeting with Sara
p:
"""

In [26]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": .1, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: Susan


#### Classification

In [27]:
prompt = f"""
Classify the topic of the following paragraph: Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group, which has a reputation for making well-timed and occasionally controversial plays in the defense industry, has quietly placed its bets on another part of the market.
Label: Business.

##

Classify the topic of the following paragraph: Some People Not Eligible to Get in on Google IPO Google has billed its IPO as a way for everyday people to get in on the process, denying Wall Street the usual stranglehold it's had on IPOs. Public bidding, a minimum of just five shares, an open process with 28 underwriters - all this pointed to a new level of public participation. But this isn't the case.
Label: Technology.

##

Classify the topic of the following paragraph: Indians Mount Charge The Cleveland Indians pulled within one game of the AL Central lead by beating the Minnesota Twins, 7-1, Saturday night with home runs by Travis Hafner and Victor Martinez.
Label: Sports.

##

Classify the topic of the following paragraph: Uptown girl, she's been living in her uptown world, I bet she never had a backstreet guy, I bet her mother never told her why, I'm gonna try.
Label:
"""

In [28]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": .1, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: Pop


In [29]:
prompt = f"""

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: My native language is not English, I have blogs and I write my own articles. I also get articles from outsource writers, so I want to use it to re-write such articles, so i will create a tool for that
NLP Task: Paraphrasing
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: Just experimenting with content generation
NLP task: Long form generation
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: My company MetaDialog provides human in the loop automated support for costumers, we are currently using GPT3 to generate answers using our custom search engine to clients questions, and then provide the answers as suggestions to human agents to use or reject them as real answers to clients.
NLP task: Conversational agent
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: Receipt extraction including line items from plain text (sources being OCR-ed images, PDFs and HTML Emails)
NLP task: Information extraction
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: I have a lot of legacy documentation which needs to be cleaned, summarized, and queried. I think AI21 would help.
NLP task: Summarization

##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: Answer questions based on a given corpus of information
NLP task: Question answering
##

Classify the below use-case description to one of the following NLP tasks: Short form generation, Long form generation, Summarization, Classification, Question answering, Paraphrasing, Conversational agent, Information extraction, Generate code

Use case: creating useful content for companies websites articles
NLP task:
"""

In [30]:
payload = {"text_inputs":prompt, "max_length": 50, "temperature": .1, "do_sample": True, "seed": 123}
response = query_endpoint_with_json_payload(url, payload)
response = parse_response_multiple_texts(response)[0]
logger.info(f'Response: {response}')

Response: Long form generation
